In [1]:
import pandas as pd
import numpy as np
import string
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eyabe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eyabe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eyabe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df=pd.read_csv("Mental_Health_FAQ.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Question_ID  98 non-null     int64 
 1   Questions    98 non-null     object
 2   Answers      98 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.4+ KB


In [4]:
df.head()

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [5]:
df=df.drop(["Question_ID"],axis="columns")

In [6]:
from nltk.tokenize import word_tokenize
df["Questions"]=df["Questions"].str.lower()

In [7]:
text=df["Questions"][0]

In [8]:
from nltk.tokenize import RegexpTokenizer
text = re.sub(r'[^\w\s]', '', text)
words = word_tokenize(text) 

In [9]:
print(words)

['what', 'does', 'it', 'mean', 'to', 'have', 'a', 'mental', 'illness']


In [10]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
text=" ".join([wnl.lemmatize(t) for t in text.split() ])

In [11]:
text

'what doe it mean to have a mental illness'

In [12]:
from nltk.corpus import stopwords  
stopwords=stopwords.words('english')

In [13]:
def data_prep(text):
  text=text.lower()
  text = re.sub(r'[^\w\s]', '', text)
  text=" ".join(t for t in text.split() if t not in stopwords)
  return text

In [14]:
print(data_prep(text))

doe mean mental illness


In [15]:
df["Questions"]=df["Questions"].apply(data_prep)

In [16]:
df["Questions"]

0               mean mental illness
1             mental illness affect
2             causes mental illness
3      warning signs mental illness
4     people mental illness recover
                  ...              
93            know im drinking much
94    cannabis dangerous legalizing
95          convince kids use drugs
96    legal status evidence cbd oil
97                  evidence vaping
Name: Questions, Length: 98, dtype: object

In [17]:
print(df["Questions"][0])

mean mental illness


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
#Tfidf vectorizer
Tfidf_vectorizer=TfidfVectorizer()
#transformed train reviews
tf_train=Tfidf_vectorizer.fit_transform(df["Questions"])


In [24]:
from sklearn.preprocessing import LabelEncoder
label_encoder= LabelEncoder()
df["Answers_Code"]=label_encoder.fit_transform(df["Answers"])

In [25]:
df["Answers_Code"]

0     48
1     35
2     36
3     72
4     91
      ..
93    69
94    18
95    96
96    17
97     0
Name: Answers_Code, Length: 98, dtype: int32

In [26]:
df

,Questions,Answers,Answers_Code
0,mean mental illness,Mental illnesses are health conditions that di...,48
1,mental illness affect,It is estimated that mental illness affects 1 ...,35
2,causes mental illness,It is estimated that mental illness affects 1 ...,36
3,warning signs mental illness,Symptoms of mental health disorders vary depen...,72
4,people mental illness recover,"When healing from mental illness, early identi...",91
...,...,...,...
93,know im drinking much,Sorting out if you are drinking too much can b...,69
94,cannabis dangerous legalizing,"Cannabis smoke, for example, contains cancer-c...",18
95,convince kids use drugs,You can't. But you can influence their capacit...,96
96,legal status evidence cbd oil,Cannabidiol or CBD is a naturally occurring co...,17


In [27]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes_model=MultinomialNB()
naive_bayes_model.fit(tf_train,df["Answers_Code"])

MultinomialNB()

In [28]:
def predict_answers(questions):
    questions_prepped = [data_prep(question) for question in questions]
    questions_vectorized = Tfidf_vectorizer.transform(questions_prepped)
    predicted_codes = naive_bayes_model.predict(questions_vectorized)
    predicted_answers = label_encoder.inverse_transform(predicted_codes)
    return predicted_answers

In [29]:
test_questions = [
    "How can someone help a friend who is experiencing depression?",
    "What is the difference between a psychiatrist and a psychologist?",
    "Can exercise help improve mental health?",
    "Where can someone find resources for mental health support?",
    "What is the difference between a psychiatrist and a psychologist?",
]

In [30]:
predicted_answers = predict_answers(test_questions)
for question, answer in zip(test_questions, predicted_answers):
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Question: How can someone help a friend who is experiencing depression?
Answer: We naturally want to help a loved one who isn’t feeling well. How we can or should help may seem fairly obvious when a loved one experiences a physical health problem, but many people say they’re not sure how to best help when a loved one experiences a mental illness like depression. Here are some tips: 
 You don’t have to understand exactly what someone else is experiencing, you just need to recognize that it’s a difficult experience. You can try something like, “I can see that this experience or these symptoms are really painful for you,” or, “I’m sorry to hear that you’re feeling unwell.” If you have experienced depression yourself, you might say, “I know how you feel.” Just remember that everyone experiences illnesses like depression differently, and empathy is about recognizing the impact of the illness, not comparing symptoms. 
 You don’t have to be an expert, but learning more about depression can he

In [33]:
import pickle

# Save the trained model
with open('./model.pkl', 'wb') as model_file:
    pickle.dump(naive_bayes_model, model_file)

# Save the vectorizer
with open('./vectorizer.pkl', 'wb') as vectorizer_file:
    pickle.dump(Tfidf_vectorizer, vectorizer_file)

# Save the label encoder
with open('./label_encoder.pkl', 'wb') as le_file:
    pickle.dump(label_encoder, le_file)


2